In [11]:
import pandas as pd
import sqlite3
import os
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [12]:
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

# Replace 'api_key.txt' with the name of your file, if different
api_key_file = 'openai_key.txt'
api_key = read_api_key(api_key_file)
os.environ['OPENAI_API_KEY'] = api_key

In [13]:
conn = sqlite3.connect('mhealth.db')

In [14]:
df = pd.read_csv('Mental_Health_Care_in_the_Last_4_Weeks.csv')
df.to_sql('mhealth.db', conn, if_exists='append', index=False)

10404

In [15]:
dburi = 'sqlite:///mhealth.db'

db = SQLDatabase.from_uri(dburi)
llm = OpenAI(temperature=0)


In [16]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [17]:
db_chain.run("How many rows is in the responses table of this db?")



> Entering new SQLDatabaseChain chain...
How many rows is in the responses table of this db?
SQLQuery: SELECT COUNT(*) FROM "mhealth.db";
SQLResult: [(20808,)]
Answer: There are 20808 rows in the responses table of this db.
> Finished chain.


' There are 20808 rows in the responses table of this db.'

In [18]:
db_chain.run("Describe the response table")



> Entering new SQLDatabaseChain chain...
Describe the response table
SQLQuery: SELECT "Indicator", "Group", "State", "Subgroup", "Phase", "Time Period", "Time Period Label", "Time Period Start Date", "Time Period End Date", "Value", "LowCI", "HighCI", "Confidence Interval", "Quartile Range", "Suppression Flag" FROM "mhealth.db" LIMIT 5;
SQLResult: [('Took Prescription Medication for Mental Health, Last 4 Weeks', 'National Estimate', 'United States', 'United States', '2', 13, 'Aug 19 - Aug 31, 2020', '08/19/2020', '08/31/2020', 19.4, 19.0, 19.8, '19.0 - 19.8', None, None), ('Took Prescription Medication for Mental Health, Last 4 Weeks', 'By Age', 'United States', '18 - 29 years', '2', 13, 'Aug 19 - Aug 31, 2020', '08/19/2020', '08/31/2020', 18.7, 17.2, 20.3, '17.2 - 20.3', None, None), ('Took Prescription Medication for Mental Health, Last 4 Weeks', 'By Age', 'United States', '30 - 39 years', '2', 13, 'Aug 19 - Aug 31, 2020', '08/19/2020', '08/31/2020', 18.3, 17.3, 19.2, '17.3 - 19.2'

' The response table contains 15 columns: Indicator, Group, State, Subgroup, Phase, Time Period, Time Period Label, Time Period Start Date, Time Period End Date, Value, LowCI, HighCI, Confidence Interval, Quartile Range, and Suppression Flag.'

In [19]:
db_chain.run("What are the top 3 countries where these responses are from?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new SQLDatabaseChain chain...
What are the top 3 countries where these responses are from?
SQLQuery: SELECT "State", COUNT(*) AS "Count" FROM "mhealth.db" GROUP BY "State" ORDER BY "Count" DESC LIMIT 3;
SQLResult: [('United States', 7344), ('Wyoming', 264), ('Wisconsin', 264)]
Answer: The top 3 countries where these responses are from are United States, Wyoming, and Wisconsin.
> Finished chain.


' The top 3 countries where these responses are from are United States, Wyoming, and Wisconsin.'

In [20]:
db_chain.run("Give me a summary of how these customers came to hear about us. What is the most common way they hear about us?")



> Entering new SQLDatabaseChain chain...
Give me a summary of how these customers came to hear about us. What is the most common way they hear about us?
SQLQuery: SELECT "Group", COUNT("Group") AS "Number of Groups" FROM "mhealth.db" GROUP BY "Group" ORDER BY "Number of Groups" DESC LIMIT 5;
SQLResult: [('By State', 13464), ('By Age', 2128), ('By Race/Hispanic ethnicity', 1520), ('By Education', 1216), ('By Sex', 608)]
Answer: The most common way customers hear about us is by State, with 13464 customers.
> Finished chain.


' The most common way customers hear about us is by State, with 13464 customers.'